In [1]:
import numpy as np
from thirdai import bolt, dataset
import os

In [5]:
%load_ext autoreload
%autoreload 2

In [10]:
def setup_module():
  if not os.path.exists("mnist"):
    os.system(
      "curl https://www.csie.ntu.edu.tw/~cjlin/libsvmtools/datasets/multiclass/mnist.bz2 --output mnist.bz2"
    )
    os.system("bzip2 -d mnist.bz2")

  if not os.path.exists("mnist.t"):
    os.system(
      "curl https://www.csie.ntu.edu.tw/~cjlin/libsvmtools/datasets/multiclass/mnist.t.bz2 --output mnist.t.bz2"
    )
    os.system("bzip2 -d mnist.t.bz2")

In [26]:
from test_dragon import test_distributed_training
from utils import gen_numpy_training_data

In [31]:
setup_module()
num_models = 16
train_data, train_labels = gen_numpy_training_data(
    n_classes=10, n_samples=1000, convert_to_bolt_dataset=False
)
test_data, test_labels = gen_numpy_training_data(
    n_classes=10, n_samples=100, convert_to_bolt_dataset=False
)
train_data, train_labels = dataset.load_bolt_svm_dataset("mnist", 250)
test_data, test_labels = dataset.load_bolt_svm_dataset("mnist.t", 250)
models = test_distributed_training(
    num_models=num_models, train_data=train_data, train_labels=train_labels, input_layer_dim=784, hidden_layer_dim=100, output_layer_dim=10, is_numpy_data=False
)

Loaded 60000 vectors from 'mnist' in 0 seconds
Loaded 10000 vectors from 'mnist.t' in 0 seconds

======================= Bolt Model =======================
input_1 (Input): dim=784
input_1 -> fc_1 (FullyConnected): dim=100, sparsity=0.2, act_func=ReLU
fc_1 -> fc_2 (FullyConnected): dim=10, sparsity=1, act_func=Softmax

======================= Bolt Model =======================
input_1 (Input): dim=784
input_1 -> fc_1 (FullyConnected): dim=100, sparsity=0.2, act_func=ReLU
fc_1 -> fc_2 (FullyConnected): dim=10, sparsity=1, act_func=Softmax

======================= Bolt Model =======================
input_1 (Input): dim=784
input_1 -> fc_1 (FullyConnected): dim=100, sparsity=0.2, act_func=ReLU
fc_1 -> fc_2 (FullyConnected): dim=10, sparsity=1, act_func=Softmax

======================= Bolt Model =======================
input_1 (Input): dim=784
input_1 -> fc_1 (FullyConnected): dim=100, sparsity=0.2, act_func=ReLU
fc_1 -> fc_2 (FullyConnected): dim=10, sparsity=1, act_func=Softmax

=======

In [32]:
for model in models:
  assert np.all(models[0].get_layer("fc_1").weights.get()== model.get_layer("fc_1").weights.get())
  assert np.all(models[0].get_layer("fc_2").weights.get()== model.get_layer("fc_2").weights.get())
  assert np.all(models[0].get_layer("fc_1").biases.get()== model.get_layer("fc_1").biases.get())
  assert np.all(models[0].get_layer("fc_2").biases.get()== model.get_layer("fc_2").biases.get())

In [33]:
def get_acc(model,test_data,test_labels):
  predict_config = (
      bolt.graph.PredictConfig.make().with_metrics(["categorical_accuracy"]).silence()
  )
  acc = model.predict(
    test_data=test_data,
    test_labels=test_labels,
    predict_config=predict_config,
  )
  return acc[0]["categorical_accuracy"]

In [34]:
for model in models:
  print(get_acc(model,test_data,test_labels))

0.9392
0.9392
0.9392
0.9392
0.9392
0.9392
0.9392
0.9392
0.9392
0.9392
0.9392
0.9392
0.9392
0.9392
0.9392
0.9392
